# CSKLD2010-2023


In [ ]:
import pandas as pd

data = pd.read_excel('FullData.xlsx')

data.to_csv('FullData.csv', index=False)


In [ ]:
data = pd.read_csv('FullData.csv')
print(data.columns)
data2=data


In [ ]:
from datetime import datetime, timedelta

data['time'] = data.apply(lambda row: (datetime.strptime(f"{row['TIMEHR']}:{row['TIMEMIN']} {row['AMPM']}", "%H:%M %p") + pd.Timedelta(hours=12)).strftime("%H:%M:%S") if row['AMPM'] == 'PM' else datetime.strptime(f"{row['TIMEHR']}:{row['TIMEMIN']} {row['AMPM']}", "%I:%M %p").strftime("%H:%M:%S"), axis=1)

data = data.drop(['TIMEHR', 'TIMEMIN', 'AMPM'], axis=1)

print(data)


In [ ]:
print(data[['time']].head())
print(data2[['TIMEHR','TIMEMIN','AMPM']].head())


In [ ]:
print(data.dtypes)


In [ ]:
data['time'] = data['time'].astype(str)
data['time'] = pd.to_datetime(data['time'], format='%H:%M:%S')

data['hour'] = data['time'].dt.hour
data['minute'] = data['time'].dt.minute

data = data.drop('time', axis=1)

print(data)


In [ ]:
print(data2.columns)


In [ ]:
databeforeNA= data
databeforeNA = databeforeNA.drop('CAUSE2', axis=1)
num_rows = databeforeNA.shape[0]
print("# of records before cleaning", num_rows)
databeforeNA = databeforeNA.dropna()
num_rows = databeforeNA.shape[0]
print("# of records after NA removal", num_rows)


In [ ]:
datawithTime=databeforeNA
print(datawithTime.head())


In [ ]:

datawithTime['RAILROAD'] = datawithTime['RAILROAD'].str.lower()
datawithTime['STATE '] = datawithTime['STATE '].str.lower()
datawithTime['CAUSE'] = datawithTime['CAUSE'].str.lower()
datawithTime['RR3'] = datawithTime['RR3'].str.lower()
datawithTime['TYPEQ'] = datawithTime['TYPEQ'].str.lower()
datawithTime['TRKCLAS'] = datawithTime['TRKCLAS'].str.lower()
datawithTime['ACCTRKCL'] = datawithTime['ACCTRKCL'].str.lower()


In [ ]:
unique_values0 = datawithTime['RAILROAD'].unique()
unique_values1 = datawithTime['STATE '].unique()
unique_values2 = datawithTime['CAUSE'].unique()
unique_values3 = datawithTime['RR3'].unique()
unique_values4 = datawithTime['TYPEQ'].unique()
unique_values5 = datawithTime['TRKCLAS'].unique()
unique_values6 = datawithTime['ACCTRKCL'].unique()

print(len(unique_values0))
print(len(unique_values1))
print(len(unique_values2))
print(len(unique_values3))
print(len(unique_values4))
print(len(unique_values5))
print(len(unique_values6))


In [ ]:
print(data.columns)


In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
cat_cols = ['RAILROAD','STATE ', 'CAUSE', 'RR3','TYPEQ','TRKCLAS','ACCTRKCL']
for col in cat_cols:
    if datawithTime[col].dtype == 'object':
        datawithTime[col] = le.fit_transform(datawithTime[col])


In [ ]:
print(datawithTime[['STATE ']].head())
print(datawithTime[['RAILROAD']].head())
print(datawithTime[['CAUSE']].head())
print(datawithTime[['RR3']].head())
print(datawithTime[['TYPEQ']].head())
print(datawithTime[['TRKCLAS']].head())
print(datawithTime[['ACCTRKCL']].head())


In [ ]:
print(datawithTime.dtypes)


In [ ]:
print(datawithTime.columns)


In [ ]:
df =  datawithTime

target_cols = ['ACCDMG', 'CASKLD', 'CASINJ', 'EVACUATE', 'CARSDMG', 'CARSHZD']

for col in target_cols:
    unique_classes = df[col].unique()
    print(f"Unique classes in {col}: {unique_classes}")


In [ ]:
df = datawithTime

df['YEAR'] = pd.to_datetime(df['YEAR'], format='%Y').dt.year

record_counts = df.groupby('YEAR').size().reset_index(name='count')

print(record_counts)


In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold
from sklearn.utils.multiclass import type_of_target

df = datawithTime

target_cols = ['CASKLD']
feature_cols = ['RAILROAD', 'RR3', 'MONTH', 'DAY', 'CARS', 'STATE ', 'TEMP', 'VISIBLTY', 'WEATHER',
                'TRNSPD', 'TONS', 'TYPEQ', 'TRKCLAS', 'TYPTRK', 'POSITON1',
                'HEADEND1', 'LOADF1', 'EMPTYF1', 'CAUSE', 'ACCTRK', 'ACCTRKCL',
                'HIGHSPD', 'hour', 'minute', 'YEAR']

cv = StratifiedKFold(n_splits=5)

for i, target_col in enumerate(target_cols):
    print(f"Processing target variable {i+1} ({target_col})")

    n_classes = len(pd.unique(df[target_col]))
    clf = RandomForestClassifier(n_estimators=100)
    clf.n_classes_ = n_classes

    for year in range(2023, 2024):
        print(f"Processing year: {year}")
        year_df = df[df['YEAR'] == year]
        X = year_df[feature_cols]
        y = year_df[target_col]

        y_type = type_of_target(y)
        if y_type not in ("binary", "multiclass"):
            raise ValueError("{0} format is not supported".format(y_type))
        rfecv = RFECV(estimator=clf, step=1, cv=cv, scoring='neg_mean_squared_error')
        rfecv.fit(X, y)

        print(f"Optimal number of features: {rfecv.n_features_}")
        print(f"Selected features: {X.columns[rfecv.support_].tolist()}")
        print(f"Feature ranking: {rfecv.ranking_}")
